In [1]:

# # Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install statsmodels
import pandas as pd
from collections import Counter
import numpy as np
from statsmodels.sandbox.stats.multicomp import multipletests


In [2]:
path_file = "D:/Hanze_Groningen/STAGE/analyse/stat/beforeGene_NonCoding_Coding_both_0_TESTS.tsv"
df = pd.read_csv(path_file, sep='\t')

In [3]:
p_value_X2_self = list(df['p_value_X2_self'])
p_value_X2 = list(df['p_value_X2'])
p_value_F = list(df['p_value_F'])
df_select = df[['gene', 'chr', 'start_position_regio', 'end_position_regio', 'counts_breast', 'counts_nonbreast',
                'p_value_X2_self', 'p_value_X2', 'p_value_F']]

In [4]:
# Create a list of the adjusted p-values --> one-step correction
bon_rej_X2_self, bon_p_adjusted_value_X2_self = multipletests(p_value_X2_self, alpha=0.05, method='bonferroni')[:2]
bon_rej_X2, bon_p_adjusted_value_X2 = multipletests(p_value_X2, alpha=0.05, method='bonferroni')[:2]
bon_rej_F, bon_p_adjusted_value_F = multipletests(p_value_F, alpha=0.05, method='bonferroni')[:2]
df_select['bon_X2_self'] = bon_p_adjusted_value_X2_self
df_select['bon_X2'] = bon_p_adjusted_value_X2
df_select['bon_F'] = bon_p_adjusted_value_F


C:\Users\Anne_\AppData\Local\Temp\ipykernel_19884\3978286935.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['bon_X2_self'] = bon_p_adjusted_value_X2_self
C:\Users\Anne_\AppData\Local\Temp\ipykernel_19884\3978286935.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['bon_X2'] = bon_p_adjusted_value_X2
C:\Users\Anne_\AppData\Local\Temp\ipykernel_19884\3978286935.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [5]:
# Create a list of the adjusted p-values --> Benjamini/Hochberg  (non-negative)
bh_rej_X2_self, bh_p_adjusted_value_X2_self = multipletests(p_value_X2_self, alpha=0.05, method='fdr_bh')[:2]
bh_rej_X2, bh_p_adjusted_value_X2 = multipletests(p_value_X2, alpha=0.05, method='fdr_bh')[:2]
bh_rej_F, bh_p_adjusted_value_F = multipletests(p_value_F, alpha=0.05, method='fdr_bh')[:2]
df_select['bh_X2_self'] = bh_p_adjusted_value_X2_self
df_select['bh_X2'] = bh_p_adjusted_value_X2
df_select['bh_F'] = bh_p_adjusted_value_F
df_select['gene_info'] = df_select['gene'].map(str) + '_' + df_select['chr'].map(str) + '_' + df_select['start_position_regio'].map(str) + '_' + df_select['end_position_regio'].map(str)

#??? 'fdr_tsbh' : two stage fdr correction (Benjamini/Hochberg)
#`fdr_i`, `fdr_bh` : Benjamini/Hochberg  (non-negative)

C:\Users\Anne_\AppData\Local\Temp\ipykernel_19884\1307992827.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['bh_X2_self'] = bh_p_adjusted_value_X2_self
C:\Users\Anne_\AppData\Local\Temp\ipykernel_19884\1307992827.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['bh_X2'] = bh_p_adjusted_value_X2
C:\Users\Anne_\AppData\Local\Temp\ipykernel_19884\1307992827.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [6]:
df_X2_self = df_select[['gene_info', 'p_value_X2_self', 'bon_X2_self', 'bh_X2_self']]
df_X2 = df_select[['gene_info','p_value_X2', 'bon_X2', 'bh_X2']]
df_F = df_select[['gene_info','p_value_F', 'bon_F', 'bh_F']]



In [7]:
def get_significant(df, test_type):
    print(f'---------{test_type}--------')
    normal_p = df[df[f'p_value_{test_type}'] <= 0.05]
    sort_normal_p = normal_p.sort_values(f'p_value_{test_type}')
    genes_normal = sort_normal_p['gene_info']
    print(f'nomal: {len(genes_normal)}')

    bon_p = df[df[f'bon_{test_type}'] <= 0.05]
    sort_bon = bon_p.sort_values(f'bon_{test_type}')
    bon_genes = sort_bon['gene_info']
    print(f'bon: {len(bon_genes)}')

    bh_p = df[df[f'bh_{test_type}'] <= 0.05]
    sort_bh = bh_p.sort_values(f'bh_{test_type}')
    bh_genes = sort_bh['gene_info']
    print(f'bh: {len(bh_genes)}')


    return genes_normal, bon_genes, bh_genes
    

In [8]:
X2_self_genes_normal, X2_self_bon_genes, X2_self_bh_genes = get_significant(df_X2_self, 'X2_self')
X2_genes_normal, X2_bon_genes, X2_bh_genes = get_significant(df_X2, 'X2')
F_genes_normal, F_bon_genes, F_bh_genes = get_significant(df_F, 'F')


---------X2_self--------
nomal: 1537
bon: 30
bh: 184
---------X2--------
nomal: 703
bon: 15
bh: 51
---------F--------
nomal: 1073
bon: 14
bh: 31


In [9]:
def print_statements(self_X2_df, X2_df, F_df):
    print('---ALL')
    all_values = set(self_X2_df) | set(X2_df) |  set(F_df)
    print(f'All genes ({len(all_values)}): {all_values}')
    elements_in_all = list(set.intersection(*map(set, [self_X2_df, X2_df, F_df])))
    print(f'Gene in all list ({len(elements_in_all)}): {elements_in_all}\n')
    # selfX2 and X2
    print('---selfX2 and X2')
    in_selfX2_X2 = set(self_X2_df) & set(X2_df)
    print(f'in selfX2 and X2 ({len(in_selfX2_X2)}): {in_selfX2_X2}')
    dif_selfX2_X2 = set(self_X2_df) - set(X2_df)
    print(f'in selfX2 NOT in X2 ({len(dif_selfX2_X2)}): {dif_selfX2_X2}')
    dif_X2_selfX2 = set(X2_df) - set(self_X2_df)
    print(f'in X2 NOT in selfx2 ({len(dif_X2_selfX2)}): {dif_X2_selfX2}\n')
    # selfX2 and F
    print('---selfX2 and F')
    in_selfX2_F = set(self_X2_df) & set(F_df)
    print(f'in selfX2 and F ({len(in_selfX2_F)}): {in_selfX2_F}')
    dif_selfX2_F = set(self_X2_df) - set(F_df)
    print(f'in selfX2 NOT in F ({len(dif_selfX2_F)}): {dif_selfX2_F}')
    dif_F_selfX2 = set(F_df) - set(self_X2_df)
    print(f'in F NOT in selfX2 ({len(dif_F_selfX2)}): {dif_F_selfX2}\n')
    # F and X2
    print('---F and X2')
    in_F_X2 = set(F_df) & set(X2_df)
    print(f'in F and X2 ({len(in_F_X2)}): {in_F_X2}')
    dif_F_X2 = set(F_df) - set(X2_df)
    print(f'in F NOT in X2 ({len(dif_F_X2)}): {dif_F_X2}')
    dif_X2_F = set(X2_df) - set(F_df)
    print(f'in X2 NOT in F ({len(dif_X2_F)}): {dif_X2_F}')

    return elements_in_all

In [10]:
top_num = 20

In [11]:
elements_in_all_normal = print_statements(X2_self_genes_normal[:top_num], X2_genes_normal[:top_num], F_genes_normal[:top_num])


---ALL
All genes (32): {'MIR223HG_X_65233302_65235552', 'CNPY4_7_99715264_99717514', 'TM4SF18_3_149034284_149036534', 'MIR190B_1_154164140_154166390', 'TAS2R19_12_11172270_11174520', 'TRBV5-4_7_142166381_142168631', 'MUC12_7_100610903_100613153', 'ATP6V0E2-AS1_7_149562782_149565032', 'GRB7_17_37893023_37895273', 'CCDC144NL-AS1_17_20769745_20771995', 'CTDSP1_2_219261060_219263310', 'LOC642236_9_68425782_68428032', 'ITGB3BP_1_63904440_63906690', 'ACR_22_51174651_51176901', 'APLP1_19_36357400_36359650', 'OR4C45_11_48364899_48367149', 'MIR5695_6_126431836_126434086', 'ERAL1_17_27180042_27182292', 'TRNA_Cys_7_149251801_149254051', 'YY1_14_100703101_100705351', 'APEX2_X_55024755_55027005', 'TRNA_Tyr_7_149253132_149255382', 'SNORA51_8_60047930_60050180', 'SMARCA1_X_128578477_128580727', 'HINT3_6_126275860_126278110', 'FANK1_10_127583107_127585357', 'AC136932.1_16_33959676_33961926', 'MUC3A_7_100548749_100550999', 'RP4-806M20.5_20_57327405_57329655', 'BV6S4-BJ2S2_7_142460183_142462433', 'OR8U8

In [12]:
elements_in_all_bon = print_statements(X2_self_bon_genes[:top_num], X2_bon_genes[:top_num], F_bon_genes[:top_num])


---ALL
All genes (25): {'OR4C45_11_48364899_48367149', 'MIR5695_6_126431836_126434086', 'ERAL1_17_27180042_27182292', 'TRNA_Cys_7_149251801_149254051', 'MIR223HG_X_65233302_65235552', 'CNPY4_7_99715264_99717514', 'YY1_14_100703101_100705351', 'TRNA_Tyr_7_149253132_149255382', 'SNORA51_8_60047930_60050180', 'SMARCA1_X_128578477_128580727', 'MIR190B_1_154164140_154166390', 'HINT3_6_126275860_126278110', 'TAS2R19_12_11172270_11174520', 'FANK1_10_127583107_127585357', 'AC136932.1_16_33959676_33961926', 'MUC3A_7_100548749_100550999', 'RP4-806M20.5_20_57327405_57329655', 'MUC12_7_100610903_100613153', 'ATP6V0E2-AS1_7_149562782_149565032', 'GRB7_17_37893023_37895273', 'GPR137C_14_53017865_53020115', 'CTDSP1_2_219261060_219263310', 'BV6S4-BJ2S2_7_142460183_142462433', 'LOC642236_9_68425782_68428032', 'ACR_22_51174651_51176901'}
Gene in all list (9): ['MIR5695_6_126431836_126434086', 'TRNA_Cys_7_149251801_149254051', 'CTDSP1_2_219261060_219263310', 'TRNA_Tyr_7_149253132_149255382', 'MUC3A_7_100

In [13]:
elements_in_all_bh = print_statements(X2_self_bh_genes[:top_num], X2_bh_genes[:top_num], F_bh_genes[:top_num])


---ALL
All genes (33): {'MIR223HG_X_65233302_65235552', 'CNPY4_7_99715264_99717514', 'RNU7-171P_9_131304194_131306444', 'MIR190B_1_154164140_154166390', 'TAS2R19_12_11172270_11174520', 'TRBV5-4_7_142166381_142168631', 'MUC12_7_100610903_100613153', 'ATP6V0E2-AS1_7_149562782_149565032', 'GRB7_17_37893023_37895273', 'GPR137C_14_53017865_53020115', 'CCDC144NL-AS1_17_20769745_20771995', 'CTDSP1_2_219261060_219263310', 'LOC642236_9_68425782_68428032', 'DYNLT3_X_37696088_37698338', 'ACR_22_51174651_51176901', 'APLP1_19_36357400_36359650', 'OR4C45_11_48364899_48367149', 'MIR5695_6_126431836_126434086', 'ERAL1_17_27180042_27182292', 'TRNA_Cys_7_149251801_149254051', 'YY1_14_100703101_100705351', 'APEX2_X_55024755_55027005', 'TRNA_Tyr_7_149253132_149255382', 'SNORA51_8_60047930_60050180', 'SMARCA1_X_128578477_128580727', 'HINT3_6_126275860_126278110', 'FANK1_10_127583107_127585357', 'AC136932.1_16_33959676_33961926', 'MUC3A_7_100548749_100550999', 'RP4-806M20.5_20_57327405_57329655', 'BV6S4-BJ2

In [14]:
path_search_snp = "D:/Hanze_Groningen/STAGE/analyse/stat/ALL_gene_before_2000_250.tsv"
snps_search = pd.read_csv(path_search_snp, sep='\t')
snps_search['gene_info'] = snps_search['gene'].map(str) + '_' + snps_search['chr'].map(str) + '_' + snps_search['start_position_regio'].map(str) + '_' + snps_search['end_position_regio'].map(str)

In [15]:
snps_normal_df = snps_search[snps_search['gene_info'].isin(elements_in_all_normal)]
snps_bon_df = snps_search[snps_search['gene_info'].isin(elements_in_all_bon)]
snps_bh_df = snps_search[snps_search['gene_info'].isin(elements_in_all_bh)]


In [16]:
snps_bh_df.columns

Index(['filter', 'gene', 'chr', 'start_position_regio', 'end_position_regio',
       '#snp_unique', 'snp_list', '#donors_all', 'donor_count', 'cancer_count',
       'gene_info'],
      dtype='object')

In [24]:
significant_snps_normal = set(','.join(list(snps_normal_df['snp_list'])).split(','))
significant_snps_bon = set(','.join(list(snps_bon_df['snp_list'])).split(','))
significant_snps_bh = set(','.join(list(snps_bh_df['snp_list'])).split(','))
print(len(significant_snps_normal))
print(len(significant_snps_bon))
print(len(significant_snps_bh))

534
510
534
